# Analisando o livro Moby Dick

In [1]:
# Inicializando uma sessão no PySpark

from pyspark.sql import SparkSession
spark = SparkSession \
          .builder \
          .appName("CDIA4-22-BookAnalysis-PySpark") \
          .getOrCreate()


spark

## Leitura do livro

In [33]:
# Lendo o arquivo .txt e visualizando-o

book = spark.read.text("./livros_para_analise/the-adventures-of-sherlock-holmes.txt")
book.show(truncate = 50)

+--------------------------------------------------+
|                                             value|
+--------------------------------------------------+
|                                                  |
|Project Gutenberg's The Adventures of Sherlock ...|
|                                                  |
|This eBook is for the use of anyone anywhere at...|
|almost no restrictions whatsoever.  You may cop...|
|re-use it under the terms of the Project Gutenb...|
|    with this eBook or online at www.gutenberg.net|
|                                                  |
|                                                  |
|          Title: The Adventures of Sherlock Holmes|
|                                                  |
|                        Author: Arthur Conan Doyle|
|                                                  |
|     Release Date: November 29, 2002 [EBook #1661]|
|                        Last Updated: May 20, 2019|
|                                             

## Tokenização

### Split

In [34]:
from pyspark.sql.functions import split
# Separando elementos por " "
lines = book.select(
    split(book.value, " ").alias("line")
)

# Visualizando
lines.show(truncate=50)

+--------------------------------------------------+
|                                              line|
+--------------------------------------------------+
|                                                []|
|[Project, Gutenberg's, The, Adventures, of, She...|
|                                                []|
|[This, eBook, is, for, the, use, of, anyone, an...|
|[almost, no, restrictions, whatsoever., , You, ...|
|[re-use, it, under, the, terms, of, the, Projec...|
|[with, this, eBook, or, online, at, www.gutenbe...|
|                                                []|
|                                                []|
|   [Title:, The, Adventures, of, Sherlock, Holmes]|
|                                                []|
|                   [Author:, Arthur, Conan, Doyle]|
|                                                []|
|[Release, Date:, November, 29,, 2002, [EBook, #...|
|                  [Last, Updated:, May, 20,, 2019]|
|                                             

### Explodindo linhas

In [35]:
# Explodindo as linhas (transformando cada elemento das listas que estão nas linhas em uma nova linha deixando cada palavra separada)

from pyspark.sql.functions import explode, col

words = lines.select(
    explode(col("line")).alias("word")
)
words.show()

+-----------+
|       word|
+-----------+
|           |
|    Project|
|Gutenberg's|
|        The|
| Adventures|
|         of|
|   Sherlock|
|    Holmes,|
|         by|
|     Arthur|
|      Conan|
|      Doyle|
|           |
|       This|
|      eBook|
|         is|
|        for|
|        the|
|        use|
|         of|
+-----------+
only showing top 20 rows



## Limpando os dados

### Deixando as palavras minúsculas

In [36]:
# Mudar a caixa das palavras para minúsculo

from pyspark.sql.functions import lower

lowered_words = words.select(
    lower(
        col("word")
    ).alias("lowered_word")
)
lowered_words.show(15)

+------------+
|lowered_word|
+------------+
|            |
|     project|
| gutenberg's|
|         the|
|  adventures|
|          of|
|    sherlock|
|     holmes,|
|          by|
|      arthur|
|       conan|
|       doyle|
|            |
|        this|
|       ebook|
+------------+
only showing top 15 rows



### Removendo a pontuação

In [37]:
from pyspark.sql.functions import regexp_extract

cleaned_words = lowered_words.select(
    regexp_extract(
        col("lowered_word"), # Indico a coluna
        "[a-z]+", # Seleciona somente o que parece uma palavra
        0 # 0 significa dar o match em toda expressão regular
    ).alias("cleaned_word")
)
cleaned_words.show()

+------------+
|cleaned_word|
+------------+
|            |
|     project|
|   gutenberg|
|         the|
|  adventures|
|          of|
|    sherlock|
|      holmes|
|          by|
|      arthur|
|       conan|
|       doyle|
|            |
|        this|
|       ebook|
|          is|
|         for|
|         the|
|         use|
|          of|
+------------+
only showing top 20 rows



### Removendo valores NULL

In [38]:
words_nonull = cleaned_words.where(
    col("cleaned_word") != ""
)
words_nonull = words_nonull.select(
    col("cleaned_word").alias("word")
)
words_nonull.show()

+----------+
|      word|
+----------+
|   project|
| gutenberg|
|       the|
|adventures|
|        of|
|  sherlock|
|    holmes|
|        by|
|    arthur|
|     conan|
|     doyle|
|      this|
|     ebook|
|        is|
|       for|
|       the|
|       use|
|        of|
|    anyone|
|  anywhere|
+----------+
only showing top 20 rows



## Contagem

### Contando a frequência de cada palavra 

In [39]:
counts = words_nonull.groupBy(
    col("word")
).count()
counts.show()

+------------+-----+
|        word|count|
+------------+-----+
|      online|    4|
|       still|   80|
|       those|   67|
|        some|  245|
|         few|   77|
|      doubts|    5|
|       inner|    5|
|        hope|   32|
|    everyday|    1|
|   connected|    7|
| requirement|    1|
|     flashed|    3|
|      travel|    3|
|  concluding|    1|
|         fog|    2|
|      spared|    2|
|         art|    5|
|accumulation|    1|
|      waters|    1|
|      poetry|    1|
+------------+-----+
only showing top 20 rows



### Ordenando a contagem

In [40]:
counts.orderBy(
    col("count"),
    ascending=False
).show()

+----+-----+
|word|count|
+----+-----+
| the| 5805|
| and| 3071|
|   i| 3029|
|  to| 2822|
|  of| 2778|
|   a| 2683|
|  in| 1819|
|that| 1757|
|  it| 1743|
| you| 1565|
|  he| 1481|
| was| 1411|
| his| 1159|
|  is| 1146|
|  my| 1006|
|have|  929|
|with|  878|
|  as|  862|
| had|  830|
|  at|  780|
+----+-----+
only showing top 20 rows



## Retomando a limpeza

In [41]:
words_lst_df = words_nonull.select(
    split(
        col("word"),
        " "
    ).alias("word")
)
words_lst_df.show()

+------------+
|        word|
+------------+
|   [project]|
| [gutenberg]|
|       [the]|
|[adventures]|
|        [of]|
|  [sherlock]|
|    [holmes]|
|        [by]|
|    [arthur]|
|     [conan]|
|     [doyle]|
|      [this]|
|     [ebook]|
|        [is]|
|       [for]|
|       [the]|
|       [use]|
|        [of]|
|    [anyone]|
|  [anywhere]|
+------------+
only showing top 20 rows



### Stop words

In [42]:
# Removendo as stop words
from pyspark.ml.feature import StopWordsRemover

swr = StopWordsRemover(
    inputCol="word",
    outputCol="meaningful"
)

swr_df = swr.transform(words_lst_df)
swr_df.show()

+------------+------------+
|        word|  meaningful|
+------------+------------+
|   [project]|   [project]|
| [gutenberg]| [gutenberg]|
|       [the]|          []|
|[adventures]|[adventures]|
|        [of]|          []|
|  [sherlock]|  [sherlock]|
|    [holmes]|    [holmes]|
|        [by]|          []|
|    [arthur]|    [arthur]|
|     [conan]|     [conan]|
|     [doyle]|     [doyle]|
|      [this]|          []|
|     [ebook]|     [ebook]|
|        [is]|          []|
|       [for]|          []|
|       [the]|          []|
|       [use]|       [use]|
|        [of]|          []|
|    [anyone]|    [anyone]|
|  [anywhere]|  [anywhere]|
+------------+------------+
only showing top 20 rows



### Removendo valores NULL

In [43]:
# Removendo os NULLs
meaningful = swr_df.select(
    (col("meaningful")[0]).alias("meaningful")
)
meaningful.show()

+----------+
|meaningful|
+----------+
|   project|
| gutenberg|
|      null|
|adventures|
|      null|
|  sherlock|
|    holmes|
|      null|
|    arthur|
|     conan|
|     doyle|
|      null|
|     ebook|
|      null|
|      null|
|      null|
|       use|
|      null|
|    anyone|
|  anywhere|
+----------+
only showing top 20 rows



In [44]:
# Filtrar os valores NULLs
meaningful_nonull = meaningful.where(
    col("meaningful") != "null"
)
meaningful_nonull.show(5)

+----------+
|meaningful|
+----------+
|   project|
| gutenberg|
|adventures|
|  sherlock|
|    holmes|
+----------+
only showing top 5 rows



### Contando os valores novamente

In [45]:
words_counts = meaningful_nonull.groupby(
    col("meaningful")
).count()

words_counts.show()

+------------+-----+
|  meaningful|count|
+------------+-----+
|      online|    4|
|       still|   80|
|      doubts|    5|
|       inner|    5|
|        hope|   32|
|    everyday|    1|
|   connected|    7|
| requirement|    1|
|     flashed|    3|
|      travel|    3|
|  concluding|    1|
|         fog|    2|
|      spared|    2|
|         art|    5|
|accumulation|    1|
|      waters|    1|
|      poetry|    1|
|    whishing|    1|
|       crest|    1|
|     implore|    2|
+------------+-----+
only showing top 20 rows



In [46]:
# Classificação da contagem
words_counts = words_counts.orderBy(
    col("count"), 
    ascending=False
)
words_counts.show()


+----------+-----+
|meaningful|count|
+----------+-----+
|      said|  486|
|    holmes|  467|
|      upon|  466|
|       one|  374|
|       man|  303|
|        mr|  274|
|    little|  269|
|       see|  232|
|       may|  213|
|      well|  199|
|        us|  184|
|     think|  174|
|      know|  171|
|     shall|  171|
|      must|  171|
|      come|  162|
|      time|  151|
|       two|  148|
|      came|  146|
|      door|  143|
+----------+-----+
only showing top 20 rows



## Análise
$Said$ - A palavra "$said$" que em português é "disse" é a palavra mais utilizada e isso se dá pelo fato do livro ser composto por muitas falas, isso nos demonstra a quantidade de detalhes que livro possui e o aprofundamento do diálogo entre os personagens, podemos concluir isso com base nas grandes resoluções de enigmas que o detetive sherlock holmes é famoso por desvendar e explicar aos demais personagens ao longo da história.

$Holmes$ - Holmes é a segunda palavra mais utilizada visto que é a forma mais comum que o personagem principal Sherlock Holmes é chamado.

$Upon$ - Já a palavra $upon$ é uma preposição utilizada para dizer "acerca de" ou até "em consequência de" utilizado nos discursos mas também pode ser utilizada para indicar uma posição ou direção, sendo também muito utilizada nos discursos com esse sentido.

$Man$ - A palavra man é muito utilizada devido as descrições de situações em que não se possuem pessoas específicas e também nas descrições de ações dos personagens.

$Mr$ - Mr é um pronome de tratamento utilizado exclusivamente para homens na língua inglesa. Isso nos mostra a predominâcia de personagens masculinos no livro nos levando ao contexto em que foi escrito, no Reino Unido do século XIX (19) entre os anos de 1891 e 1892, ou seja, os contos se passam num antigo Reino Unido e retrata inclusive a infame Ku Klux Klan (KKK) no conto “As Cinco Sementes de Laranja”.